# 데이터 불러오기

In [1]:
pip install pytimekr

  Preparing metadata (setup.py) ... done
  Created wheel for pytimekr: filename=pytimekr-0.1.0-py3-none-any.whl size=7925 sha256=b0280f765f00c70d5dba0bbb545f419407a97ab80d3132aa35905db5ccfad9d4
  Stored in directory: /root/.cache/pip/wheels/b3/a3/ba/e73aaae1dec550b8667510c379b019295bba855275258745de
Successfully built pytimekr


In [39]:
import pandas as pd
import numpy as np
from pytimekr import pytimekr
import datetime
from tqdm import tqdm

In [40]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [43]:
cd /content/drive/MyDrive/Data/power/

/content/drive/MyDrive/Data/power


# 데이터 전처리
---

### train.csv & test.csv

In [44]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
train

,num_date_time,건물번호,일시,기온(C),강수량(mm),풍속(m/s),습도(%),일조(hr),일사(MJ/m2),전력소비량(kWh)
0,1_20220601 00,1,20220601 00,18.6,NaN,0.9,42.0,NaN,NaN,1085.28
1,1_20220601 01,1,20220601 01,18.0,NaN,1.1,45.0,NaN,NaN,1047.36
2,1_20220601 02,1,20220601 02,17.7,NaN,1.5,45.0,NaN,NaN,974.88
3,1_20220601 03,1,20220601 03,16.7,NaN,1.4,48.0,NaN,NaN,953.76
4,1_20220601 04,1,20220601 04,18.4,NaN,2.8,43.0,NaN,NaN,986.40
...,...,...,...,...,...,...,...,...,...,...
203995,100_20220824 19,100,20220824 19,23.1,NaN,0.9,86.0,0.5,NaN,881.04
203996,100_20220824 20,100,20220824 20,22.4,NaN,1.3,86.0,0.0,NaN,798.96
203997,100_20220824 21,100,20220824 21,21.3,NaN,1.0,92.0,NaN,NaN,825.12
203998,100_20220824 22,100,20220824 22,21.0,NaN,0.3,94.0,NaN,NaN,640.08


In [8]:
test.head()

,num_date_time,건물번호,일시,기온(C),강수량(mm),풍속(m/s),습도(%)
0,1_20220825 00,1,20220825 00,23.5,0.0,2.2,72
1,1_20220825 01,1,20220825 01,23.0,0.0,0.9,72
2,1_20220825 02,1,20220825 02,22.7,0.0,1.5,75
3,1_20220825 03,1,20220825 03,22.1,0.0,1.3,78
4,1_20220825 04,1,20220825 04,21.8,0.0,1.0,77


* 컬럼명 영어로 변환

In [45]:
train = train.rename(columns={
    '건물번호': 'building_number',
    '일시': 'date_time',
    '기온(C)': 'temperature',
    '강수량(mm)': 'rainfall',
    '풍속(m/s)': 'windspeed',
    '습도(%)': 'humidity',
    '일조(hr)': 'sunshine',
    '일사(MJ/m2)': 'solar_radiation',
    '전력소비량(kWh)': 'power_consumption'
})
test = test.rename(columns={
    '건물번호': 'building_number',
    '일시': 'date_time',
    '기온(C)': 'temperature',
    '강수량(mm)': 'rainfall',
    '풍속(m/s)': 'windspeed',
    '습도(%)': 'humidity'
})
train

,num_date_time,building_number,date_time,temperature,rainfall,windspeed,humidity,sunshine,solar_radiation,power_consumption
0,1_20220601 00,1,20220601 00,18.6,NaN,0.9,42.0,NaN,NaN,1085.28
1,1_20220601 01,1,20220601 01,18.0,NaN,1.1,45.0,NaN,NaN,1047.36
2,1_20220601 02,1,20220601 02,17.7,NaN,1.5,45.0,NaN,NaN,974.88
3,1_20220601 03,1,20220601 03,16.7,NaN,1.4,48.0,NaN,NaN,953.76
4,1_20220601 04,1,20220601 04,18.4,NaN,2.8,43.0,NaN,NaN,986.40
...,...,...,...,...,...,...,...,...,...,...
203995,100_20220824 19,100,20220824 19,23.1,NaN,0.9,86.0,0.5,NaN,881.04
203996,100_20220824 20,100,20220824 20,22.4,NaN,1.3,86.0,0.0,NaN,798.96
203997,100_20220824 21,100,20220824 21,21.3,NaN,1.0,92.0,NaN,NaN,825.12
203998,100_20220824 22,100,20220824 22,21.0,NaN,0.3,94.0,NaN,NaN,640.08


* 일시 -> 월, 일, 요일, 시간으로 분리

In [46]:
train['date_time'] = pd.to_datetime(train['date_time'], format = '%Y%m%d %H')
test['date_time'] = pd.to_datetime(test['date_time'], format = '%Y%m%d %H')

train['day'] = train['date_time'].dt.day
train['month'] = train['date_time'].dt.month
train['day_of_week'] = train['date_time'].dt.weekday
train['hour'] = train['date_time'].dt.hour

test['day'] = test['date_time'].dt.day
test['month'] = test['date_time'].dt.month
test['day_of_week'] = test['date_time'].dt.weekday
test['hour'] = test['date_time'].dt.hour

train

,num_date_time,building_number,date_time,temperature,rainfall,windspeed,humidity,sunshine,solar_radiation,power_consumption,day,month,day_of_week,hour
0,1_20220601 00,1,2022-06-01 00:00:00,18.6,NaN,0.9,42.0,NaN,NaN,1085.28,1,6,2,0
1,1_20220601 01,1,2022-06-01 01:00:00,18.0,NaN,1.1,45.0,NaN,NaN,1047.36,1,6,2,1
2,1_20220601 02,1,2022-06-01 02:00:00,17.7,NaN,1.5,45.0,NaN,NaN,974.88,1,6,2,2
3,1_20220601 03,1,2022-06-01 03:00:00,16.7,NaN,1.4,48.0,NaN,NaN,953.76,1,6,2,3
4,1_20220601 04,1,2022-06-01 04:00:00,18.4,NaN,2.8,43.0,NaN,NaN,986.40,1,6,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203995,100_20220824 19,100,2022-08-24 19:00:00,23.1,NaN,0.9,86.0,0.5,NaN,881.04,24,8,2,19
203996,100_20220824 20,100,2022-08-24 20:00:00,22.4,NaN,1.3,86.0,0.0,NaN,798.96,24,8,2,20
203997,100_20220824 21,100,2022-08-24 21:00:00,21.3,NaN,1.0,92.0,NaN,NaN,825.12,24,8,2,21
203998,100_20220824 22,100,2022-08-24 22:00:00,21.0,NaN,0.3,94.0,NaN,NaN,640.08,24,8,2,22


#### 파생변수

* sin, cos 시간 추가 : 순환성

In [47]:
train['cos_time'] = np.cos(2 * np.pi * (train['hour']/24))
train['sin_time'] = np.sin(2 * np.pi * (train['hour']/24))

test['cos_time'] = np.cos(2 * np.pi * (test['hour']/24))
test['sin_time'] = np.sin(2 * np.pi * (test['hour']/24))
train

,num_date_time,building_number,date_time,temperature,rainfall,windspeed,humidity,sunshine,solar_radiation,power_consumption,day,month,day_of_week,hour,cos_time,sin_time
0,1_20220601 00,1,2022-06-01 00:00:00,18.6,NaN,0.9,42.0,NaN,NaN,1085.28,1,6,2,0,1.000000,0.000000
1,1_20220601 01,1,2022-06-01 01:00:00,18.0,NaN,1.1,45.0,NaN,NaN,1047.36,1,6,2,1,0.965926,0.258819
2,1_20220601 02,1,2022-06-01 02:00:00,17.7,NaN,1.5,45.0,NaN,NaN,974.88,1,6,2,2,0.866025,0.500000
3,1_20220601 03,1,2022-06-01 03:00:00,16.7,NaN,1.4,48.0,NaN,NaN,953.76,1,6,2,3,0.707107,0.707107
4,1_20220601 04,1,2022-06-01 04:00:00,18.4,NaN,2.8,43.0,NaN,NaN,986.40,1,6,2,4,0.500000,0.866025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203995,100_20220824 19,100,2022-08-24 19:00:00,23.1,NaN,0.9,86.0,0.5,NaN,881.04,24,8,2,19,0.258819,-0.965926
203996,100_20220824 20,100,2022-08-24 20:00:00,22.4,NaN,1.3,86.0,0.0,NaN,798.96,24,8,2,20,0.500000,-0.866025
203997,100_20220824 21,100,2022-08-24 21:00:00,21.3,NaN,1.0,92.0,NaN,NaN,825.12,24,8,2,21,0.707107,-0.707107
203998,100_20220824 22,100,2022-08-24 22:00:00,21.0,NaN,0.3,94.0,NaN,NaN,640.08,24,8,2,22,0.866025,-0.500000


* 공휴일(주말 포함) 여부(True/False) 분류

In [12]:
train['date_time'] = pd.to_datetime(train['date_time'], format = '%Y-%m-%d')
test['date_time'] = pd.to_datetime(test['date_time'], format = '%Y-%m-%d')

holidays = pytimekr.holidays(2022)
holiday = list()
for h in holidays:
  holiday.append(str(h))
holiday

['2022-01-31',
 '2022-02-01',
 '2022-02-02',
 '2022-09-09',
 '2022-09-10',
 '2022-09-11',
 '2022-01-01',
 '2022-03-01',
 '2022-05-05',
 '2022-05-08',
 '2022-06-06',
 '2022-08-15',
 '2022-10-09',
 '2022-10-03',
 '2022-12-25']

In [13]:
def is_holiday(d):
  d = str(d)[:10]
  if d in holiday:
    return True
  else:
    return False

def is_weekend(d):
  d = str(d)[:10]
  d = datetime.datetime.strptime(d, '%Y-%m-%d')
  d = d.weekday()
  if d > 4:
    return True
  else:
    return False

train['holiday'] = [True if(is_holiday(d) or is_weekend(d)) else False for d in train['date_time']]
test['holiday'] = [True if(is_holiday(d) or is_weekend(d)) else False for d in test['date_time']]
train

,num_date_time,building_number,date_time,temperature,rainfall,windspeed,humidity,sunshine,solar_radiation,power_consumption,day,month,day_of_week,hour,cos_time,sin_time,holiday
0,1_20220601 00,1,2022-06-01 00:00:00,18.6,NaN,0.9,42.0,NaN,NaN,1085.28,1,6,2,0,1.000000,0.000000,False
1,1_20220601 01,1,2022-06-01 01:00:00,18.0,NaN,1.1,45.0,NaN,NaN,1047.36,1,6,2,1,0.965926,0.258819,False
2,1_20220601 02,1,2022-06-01 02:00:00,17.7,NaN,1.5,45.0,NaN,NaN,974.88,1,6,2,2,0.866025,0.500000,False
3,1_20220601 03,1,2022-06-01 03:00:00,16.7,NaN,1.4,48.0,NaN,NaN,953.76,1,6,2,3,0.707107,0.707107,False
4,1_20220601 04,1,2022-06-01 04:00:00,18.4,NaN,2.8,43.0,NaN,NaN,986.40,1,6,2,4,0.500000,0.866025,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203995,100_20220824 19,100,2022-08-24 19:00:00,23.1,NaN,0.9,86.0,0.5,NaN,881.04,24,8,2,19,0.258819,-0.965926,False
203996,100_20220824 20,100,2022-08-24 20:00:00,22.4,NaN,1.3,86.0,0.0,NaN,798.96,24,8,2,20,0.500000,-0.866025,False
203997,100_20220824 21,100,2022-08-24 21:00:00,21.3,NaN,1.0,92.0,NaN,NaN,825.12,24,8,2,21,0.707107,-0.707107,False
203998,100_20220824 22,100,2022-08-24 22:00:00,21.0,NaN,0.3,94.0,NaN,NaN,640.08,24,8,2,22,0.866025,-0.500000,False


In [14]:
train.loc[(train['month'] == 6) & (train['day'] == 6)]

,num_date_time,building_number,date_time,temperature,rainfall,windspeed,humidity,sunshine,solar_radiation,power_consumption,day,month,day_of_week,hour,cos_time,sin_time,holiday
120,1_20220606 00,1,2022-06-06 00:00:00,20.2,NaN,1.9,74.0,NaN,NaN,1108.32,6,6,0,0,1.000000,0.000000,True
121,1_20220606 01,1,2022-06-06 01:00:00,20.2,NaN,2.2,73.0,NaN,NaN,1062.24,6,6,0,1,0.965926,0.258819,True
122,1_20220606 02,1,2022-06-06 02:00:00,19.9,NaN,2.0,76.0,NaN,NaN,997.44,6,6,0,2,0.866025,0.500000,True
123,1_20220606 03,1,2022-06-06 03:00:00,19.3,1.3,1.5,84.0,NaN,NaN,964.32,6,6,0,3,0.707107,0.707107,True
124,1_20220606 04,1,2022-06-06 04:00:00,18.8,0.0,2.1,84.0,NaN,NaN,1003.20,6,6,0,4,0.500000,0.866025,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202099,100_20220606 19,100,2022-06-06 19:00:00,21.9,NaN,2.1,63.0,0.4,NaN,767.28,6,6,0,19,0.258819,-0.965926,True
202100,100_20220606 20,100,2022-06-06 20:00:00,20.6,NaN,1.5,68.0,0.0,NaN,753.36,6,6,0,20,0.500000,-0.866025,True
202101,100_20220606 21,100,2022-06-06 21:00:00,20.1,NaN,0.8,70.0,NaN,NaN,675.60,6,6,0,21,0.707107,-0.707107,True
202102,100_20220606 22,100,2022-06-06 22:00:00,19.6,NaN,0.7,72.0,NaN,NaN,584.16,6,6,0,22,0.866025,-0.500000,True


In [15]:
train['holiday'].unique()

array([False,  True])

* 건물별, 요일별, 시간대별 전력사용량 평균

In [17]:
train_power_mean = pd.pivot_table(train, values = 'power_consumption', index = ['building_number', 'hour', 'day'], aggfunc = np.mean).reset_index()
tqdm.pandas()
train['day_hour_mean'] = train.progress_apply(lambda x : train_power_mean.loc[(train_power_mean.building_number == x['building_number']) & (train_power_mean.hour == x['hour']) & (train_power_mean.day == x['day']) ,'power_consumption'].values[0], axis = 1)

test_power_mean = pd.pivot_table(train, values = 'power_consumption', index = ['building_number', 'hour', 'day'], aggfunc = np.mean).reset_index()
tqdm.pandas()
test['day_hour_mean'] = test.progress_apply(lambda x : test_power_mean.loc[(test_power_mean.building_number == x['building_number']) & (test_power_mean.hour == x['hour']) & (test_power_mean.day == x['day']) ,'power_consumption'].values[0], axis = 1)

train

100%|██████████| 16800/16800 [00:18<00:00, 931.01it/s] 


,num_date_time,building_number,date_time,temperature,rainfall,windspeed,humidity,sunshine,solar_radiation,power_consumption,day,month,day_of_week,hour,cos_time,sin_time,holiday,day_hour_mean
0,1_20220601 00,1,2022-06-01 00:00:00,18.6,NaN,0.9,42.0,NaN,NaN,1085.28,1,6,2,0,1.000000,0.000000,False,1371.52
1,1_20220601 01,1,2022-06-01 01:00:00,18.0,NaN,1.1,45.0,NaN,NaN,1047.36,1,6,2,1,0.965926,0.258819,False,1323.52
2,1_20220601 02,1,2022-06-01 02:00:00,17.7,NaN,1.5,45.0,NaN,NaN,974.88,1,6,2,2,0.866025,0.500000,False,1236.48
3,1_20220601 03,1,2022-06-01 03:00:00,16.7,NaN,1.4,48.0,NaN,NaN,953.76,1,6,2,3,0.707107,0.707107,False,1189.92
4,1_20220601 04,1,2022-06-01 04:00:00,18.4,NaN,2.8,43.0,NaN,NaN,986.40,1,6,2,4,0.500000,0.866025,False,1210.08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203995,100_20220824 19,100,2022-08-24 19:00:00,23.1,NaN,0.9,86.0,0.5,NaN,881.04,24,8,2,19,0.258819,-0.965926,False,942.96
203996,100_20220824 20,100,2022-08-24 20:00:00,22.4,NaN,1.3,86.0,0.0,NaN,798.96,24,8,2,20,0.500000,-0.866025,False,855.84
203997,100_20220824 21,100,2022-08-24 21:00:00,21.3,NaN,1.0,92.0,NaN,NaN,825.12,24,8,2,21,0.707107,-0.707107,False,809.20
203998,100_20220824 22,100,2022-08-24 22:00:00,21.0,NaN,0.3,94.0,NaN,NaN,640.08,24,8,2,22,0.866025,-0.500000,False,696.08


* 건물별, 시간별 전력소비량 평균

In [18]:
train_power_hour_mean = pd.pivot_table(train, values = 'power_consumption', index = ['building_number', 'hour'], aggfunc = np.mean).reset_index()
tqdm.pandas()
train['hour_mean'] = train.progress_apply(lambda x : train_power_hour_mean.loc[(train_power_hour_mean.building_number == x['building_number']) & (train_power_hour_mean.hour == x['hour']) ,'power_consumption'].values[0], axis = 1)

test_power_hour_mean = pd.pivot_table(train, values = 'power_consumption', index = ['building_number', 'hour'], aggfunc = np.mean).reset_index()
tqdm.pandas()
test['hour_mean'] = test.progress_apply(lambda x : test_power_hour_mean.loc[(test_power_hour_mean.building_number == x['building_number']) & (test_power_hour_mean.hour == x['hour']) ,'power_consumption'].values[0], axis = 1)

train

100%|██████████| 16800/16800 [00:09<00:00, 1711.47it/s]


,num_date_time,building_number,date_time,temperature,rainfall,windspeed,humidity,sunshine,solar_radiation,power_consumption,day,month,day_of_week,hour,cos_time,sin_time,holiday,day_hour_mean,hour_mean
0,1_20220601 00,1,2022-06-01 00:00:00,18.6,NaN,0.9,42.0,NaN,NaN,1085.28,1,6,2,0,1.000000,0.000000,False,1371.52,1706.318118
1,1_20220601 01,1,2022-06-01 01:00:00,18.0,NaN,1.1,45.0,NaN,NaN,1047.36,1,6,2,1,0.965926,0.258819,False,1323.52,1622.620235
2,1_20220601 02,1,2022-06-01 02:00:00,17.7,NaN,1.5,45.0,NaN,NaN,974.88,1,6,2,2,0.866025,0.500000,False,1236.48,1506.971294
3,1_20220601 03,1,2022-06-01 03:00:00,16.7,NaN,1.4,48.0,NaN,NaN,953.76,1,6,2,3,0.707107,0.707107,False,1189.92,1437.365647
4,1_20220601 04,1,2022-06-01 04:00:00,18.4,NaN,2.8,43.0,NaN,NaN,986.40,1,6,2,4,0.500000,0.866025,False,1210.08,1447.321412
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203995,100_20220824 19,100,2022-08-24 19:00:00,23.1,NaN,0.9,86.0,0.5,NaN,881.04,24,8,2,19,0.258819,-0.965926,False,942.96,1010.462118
203996,100_20220824 20,100,2022-08-24 20:00:00,22.4,NaN,1.3,86.0,0.0,NaN,798.96,24,8,2,20,0.500000,-0.866025,False,855.84,928.125176
203997,100_20220824 21,100,2022-08-24 21:00:00,21.3,NaN,1.0,92.0,NaN,NaN,825.12,24,8,2,21,0.707107,-0.707107,False,809.20,830.032941
203998,100_20220824 22,100,2022-08-24 22:00:00,21.0,NaN,0.3,94.0,NaN,NaN,640.08,24,8,2,22,0.866025,-0.500000,False,696.08,723.100235


* 건물별, 시간별 전력소비량 표준편차

In [19]:
train_power_hour_std = pd.pivot_table(train, values = 'power_consumption', index = ['building_number', 'hour'], aggfunc = np.std).reset_index()
tqdm.pandas()
train['hour_std'] = train.progress_apply(lambda x : train_power_hour_std.loc[(train_power_hour_std.building_number == x['building_number']) & (train_power_hour_std.hour == x['hour']) ,'power_consumption'].values[0], axis = 1)

test_power_hour_std = pd.pivot_table(train, values = 'power_consumption', index = ['building_number', 'hour'], aggfunc = np.std).reset_index()
tqdm.pandas()
test['hour_std'] = test.progress_apply(lambda x : test_power_hour_std.loc[(test_power_hour_std.building_number == x['building_number']) & (test_power_hour_std.hour == x['hour']) ,'power_consumption'].values[0], axis = 1)

100%|██████████| 16800/16800 [00:09<00:00, 1712.72it/s]


* 불쾌지수
---
불쾌지수 = 0.81 * temperature + 0.01 * humidity * (0.99 * temperature - 14.3) + 46.3



In [20]:
train['discomfort'] = 0.81 * train['temperature'] + 0.01 * train['humidity'] * (0.99 * train['temperature'] - 14.3) + 46.3
test['discomfort'] = 0.81 * test['temperature'] + 0.01 * test['humidity'] * (0.99 * test['temperature'] - 14.3) + 46.3
train

,num_date_time,building_number,date_time,temperature,rainfall,windspeed,humidity,sunshine,solar_radiation,power_consumption,...,month,day_of_week,hour,cos_time,sin_time,holiday,day_hour_mean,hour_mean,hour_std,discomfort
0,1_20220601 00,1,2022-06-01 00:00:00,18.6,NaN,0.9,42.0,NaN,NaN,1085.28,...,6,2,0,1.000000,0.000000,False,1371.52,1706.318118,446.882767,63.09388
1,1_20220601 01,1,2022-06-01 01:00:00,18.0,NaN,1.1,45.0,NaN,NaN,1047.36,...,6,2,1,0.965926,0.258819,False,1323.52,1622.620235,439.662704,62.46400
2,1_20220601 02,1,2022-06-01 02:00:00,17.7,NaN,1.5,45.0,NaN,NaN,974.88,...,6,2,2,0.866025,0.500000,False,1236.48,1506.971294,412.071906,62.08735
3,1_20220601 03,1,2022-06-01 03:00:00,16.7,NaN,1.4,48.0,NaN,NaN,953.76,...,6,2,3,0.707107,0.707107,False,1189.92,1437.365647,391.205981,60.89884
4,1_20220601 04,1,2022-06-01 04:00:00,18.4,NaN,2.8,43.0,NaN,NaN,986.40,...,6,2,4,0.500000,0.866025,False,1210.08,1447.321412,381.099697,62.88788
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203995,100_20220824 19,100,2022-08-24 19:00:00,23.1,NaN,0.9,86.0,0.5,NaN,881.04,...,8,2,19,0.258819,-0.965926,False,942.96,1010.462118,161.399578,72.38034
203996,100_20220824 20,100,2022-08-24 20:00:00,22.4,NaN,1.3,86.0,0.0,NaN,798.96,...,8,2,20,0.500000,-0.866025,False,855.84,928.125176,137.566008,71.21736
203997,100_20220824 21,100,2022-08-24 21:00:00,21.3,NaN,1.0,92.0,NaN,NaN,825.12,...,8,2,21,0.707107,-0.707107,False,809.20,830.032941,128.300189,69.79704
203998,100_20220824 22,100,2022-08-24 22:00:00,21.0,NaN,0.3,94.0,NaN,NaN,640.08,...,8,2,22,0.866025,-0.500000,False,696.08,723.100235,112.464079,69.41060


* 냉방도일(CDH, Cooling Degree Hour)

In [21]:
def CDH(xs):
    ys = []
    for i in range(len(xs)):
        if i < 11:
            ys.append(np.sum(xs[:(i+1)]-26))
        else:
            ys.append(np.sum(xs[(i-11):(i+1)]-26))
    return np.array(ys)

train_cdhs = np.array([])
test_cdhs = np.array([])

for num in range(1,101,1):
    train_temp = train[train['building_number'] == num]
    test_temp = test[test['building_number'] == num]
    train_cdh = CDH(train_temp['temperature'].values)
    test_cdh = CDH(test_temp['temperature'].values)
    train_cdhs = np.concatenate([train_cdhs, train_cdh])
    test_cdhs = np.concatenate([test_cdhs, test_cdh])
train['CDH'] = train_cdhs
test['CDH'] = test_cdhs
train

,num_date_time,building_number,date_time,temperature,rainfall,windspeed,humidity,sunshine,solar_radiation,power_consumption,...,day_of_week,hour,cos_time,sin_time,holiday,day_hour_mean,hour_mean,hour_std,discomfort,CDH
0,1_20220601 00,1,2022-06-01 00:00:00,18.6,NaN,0.9,42.0,NaN,NaN,1085.28,...,2,0,1.000000,0.000000,False,1371.52,1706.318118,446.882767,63.09388,-7.4
1,1_20220601 01,1,2022-06-01 01:00:00,18.0,NaN,1.1,45.0,NaN,NaN,1047.36,...,2,1,0.965926,0.258819,False,1323.52,1622.620235,439.662704,62.46400,-15.4
2,1_20220601 02,1,2022-06-01 02:00:00,17.7,NaN,1.5,45.0,NaN,NaN,974.88,...,2,2,0.866025,0.500000,False,1236.48,1506.971294,412.071906,62.08735,-23.7
3,1_20220601 03,1,2022-06-01 03:00:00,16.7,NaN,1.4,48.0,NaN,NaN,953.76,...,2,3,0.707107,0.707107,False,1189.92,1437.365647,391.205981,60.89884,-33.0
4,1_20220601 04,1,2022-06-01 04:00:00,18.4,NaN,2.8,43.0,NaN,NaN,986.40,...,2,4,0.500000,0.866025,False,1210.08,1447.321412,381.099697,62.88788,-40.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203995,100_20220824 19,100,2022-08-24 19:00:00,23.1,NaN,0.9,86.0,0.5,NaN,881.04,...,2,19,0.258819,-0.965926,False,942.96,1010.462118,161.399578,72.38034,-19.6
203996,100_20220824 20,100,2022-08-24 20:00:00,22.4,NaN,1.3,86.0,0.0,NaN,798.96,...,2,20,0.500000,-0.866025,False,855.84,928.125176,137.566008,71.21736,-20.2
203997,100_20220824 21,100,2022-08-24 21:00:00,21.3,NaN,1.0,92.0,NaN,NaN,825.12,...,2,21,0.707107,-0.707107,False,809.20,830.032941,128.300189,69.79704,-22.3
203998,100_20220824 22,100,2022-08-24 22:00:00,21.0,NaN,0.3,94.0,NaN,NaN,640.08,...,2,22,0.866025,-0.500000,False,696.08,723.100235,112.464079,69.41060,-25.1


In [22]:
test.columns

Index(['num_date_time', 'building_number', 'date_time', 'temperature',
       'rainfall', 'windspeed', 'humidity', 'day', 'month', 'day_of_week',
       'hour', 'cos_time', 'sin_time', 'holiday', 'day_hour_mean', 'hour_mean',
       'hour_std', 'discomfort', 'CDH'],
      dtype='object')

* 'num_date_time', '일시', '시간', '강수량(mm)', '일조(hr)', '일사(MJ/m2)' 열 제거

In [23]:
train = train.drop(['num_date_time', 'date_time', 'hour', 'rainfall', 'sunshine', 'solar_radiation'], axis = 1)
test = test.drop(['num_date_time', 'date_time', 'hour', 'rainfall'], axis = 1)
train

,building_number,temperature,windspeed,humidity,power_consumption,day,month,day_of_week,cos_time,sin_time,holiday,day_hour_mean,hour_mean,hour_std,discomfort,CDH
0,1,18.6,0.9,42.0,1085.28,1,6,2,1.000000,0.000000,False,1371.52,1706.318118,446.882767,63.09388,-7.4
1,1,18.0,1.1,45.0,1047.36,1,6,2,0.965926,0.258819,False,1323.52,1622.620235,439.662704,62.46400,-15.4
2,1,17.7,1.5,45.0,974.88,1,6,2,0.866025,0.500000,False,1236.48,1506.971294,412.071906,62.08735,-23.7
3,1,16.7,1.4,48.0,953.76,1,6,2,0.707107,0.707107,False,1189.92,1437.365647,391.205981,60.89884,-33.0
4,1,18.4,2.8,43.0,986.40,1,6,2,0.500000,0.866025,False,1210.08,1447.321412,381.099697,62.88788,-40.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203995,100,23.1,0.9,86.0,881.04,24,8,2,0.258819,-0.965926,False,942.96,1010.462118,161.399578,72.38034,-19.6
203996,100,22.4,1.3,86.0,798.96,24,8,2,0.500000,-0.866025,False,855.84,928.125176,137.566008,71.21736,-20.2
203997,100,21.3,1.0,92.0,825.12,24,8,2,0.707107,-0.707107,False,809.20,830.032941,128.300189,69.79704,-22.3
203998,100,21.0,0.3,94.0,640.08,24,8,2,0.866025,-0.500000,False,696.08,723.100235,112.464079,69.41060,-25.1


In [24]:
train[['building_number']] = train[['building_number']].astype('category')
test[['building_number']] = test[['building_number']].astype('category')

In [25]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16800 entries, 0 to 16799
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   building_number  16800 non-null  category
 1   temperature      16800 non-null  float64 
 2   windspeed        16800 non-null  float64 
 3   humidity         16800 non-null  int64   
 4   day              16800 non-null  int64   
 5   month            16800 non-null  int64   
 6   day_of_week      16800 non-null  int64   
 7   cos_time         16800 non-null  float64 
 8   sin_time         16800 non-null  float64 
 9   holiday          16800 non-null  bool    
 10  day_hour_mean    16800 non-null  float64 
 11  hour_mean        16800 non-null  float64 
 12  hour_std         16800 non-null  float64 
 13  discomfort       16800 non-null  float64 
 14  CDH              16800 non-null  float64 
dtypes: bool(1), category(1), float64(9), int64(4)
memory usage: 1.7 MB


In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 204000 entries, 0 to 203999
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype   
---  ------             --------------   -----   
 0   building_number    204000 non-null  category
 1   temperature        204000 non-null  float64 
 2   windspeed          203981 non-null  float64 
 3   humidity           203991 non-null  float64 
 4   power_consumption  204000 non-null  float64 
 5   day                204000 non-null  int64   
 6   month              204000 non-null  int64   
 7   day_of_week        204000 non-null  int64   
 8   cos_time           204000 non-null  float64 
 9   sin_time           204000 non-null  float64 
 10  holiday            204000 non-null  bool    
 11  day_hour_mean      204000 non-null  float64 
 12  hour_mean          204000 non-null  float64 
 13  hour_std           204000 non-null  float64 
 14  discomfort         203991 non-null  float64 
 15  CDH                204000 non-null

### building_info.csv

In [26]:
building = pd.read_csv("building_info.csv")
building

,건물번호,건물유형,연면적(m2),냉방면적(m2),태양광용량(kW),ESS저장용량(kWh),PCS용량(kW)
0,1,건물기타,110634.00,39570.00,-,-,-
1,2,건물기타,122233.47,99000.00,-,-,-
2,3,건물기타,171243.00,113950.00,40,-,-
3,4,건물기타,74312.98,34419.62,60,-,-
4,5,건물기타,205884.00,150000.00,-,2557,1000
...,...,...,...,...,...,...,...
95,96,호텔및리조트,93314.00,60500.00,-,-,-
96,97,호텔및리조트,55144.67,25880.00,-,-,-
97,98,호텔및리조트,53578.62,17373.75,-,-,-
98,99,호텔및리조트,53499.00,40636.00,-,-,-


* 컬럼명 영어로 변환

In [27]:
building = building.rename(columns={
    '건물번호': 'building_number',
    '건물유형': 'building_type',
    '연면적(m2)': 'total_area',
    '냉방면적(m2)': 'cooling_area',
    '태양광용량(kW)': 'solar_power_capacity',
    'ESS저장용량(kWh)': 'ess_capacity',
    'PCS용량(kW)': 'pcs_capacity'
})

* 건물 타입 영문 매핑

In [28]:
translation_dict = {
    '건물기타': 'Other Buildings',
    '공공': 'Public',
    '대학교': 'University',
    '데이터센터': 'Data Center',
    '백화점및아울렛': 'Department Store and Outlet',
    '병원': 'Hospital',
    '상용': 'Commercial',
    '아파트': 'Apartment',
    '연구소': 'Research Institute',
    '지식산업센터': 'Knowledge Industry Center',
    '할인마트': 'Discount Mart',
    '호텔및리조트': 'Hotel and Resort'
}

building['building_type'] = building['building_type'].replace(translation_dict)
building

,building_number,building_type,total_area,cooling_area,solar_power_capacity,ess_capacity,pcs_capacity
0,1,Other Buildings,110634.00,39570.00,-,-,-
1,2,Other Buildings,122233.47,99000.00,-,-,-
2,3,Other Buildings,171243.00,113950.00,40,-,-
3,4,Other Buildings,74312.98,34419.62,60,-,-
4,5,Other Buildings,205884.00,150000.00,-,2557,1000
...,...,...,...,...,...,...,...
95,96,Hotel and Resort,93314.00,60500.00,-,-,-
96,97,Hotel and Resort,55144.67,25880.00,-,-,-
97,98,Hotel and Resort,53578.62,17373.75,-,-,-
98,99,Hotel and Resort,53499.00,40636.00,-,-,-


* 결측치 처리

In [29]:
len(building.loc[building['solar_power_capacity'] == '-'])

64

In [30]:
len(building.loc[building['ess_capacity'] == '-'])

95

In [31]:
len(building.loc[building['pcs_capacity'] == '-'])

95

In [32]:
building.replace('-', np.nan)
building

,building_number,building_type,total_area,cooling_area,solar_power_capacity,ess_capacity,pcs_capacity
0,1,Other Buildings,110634.00,39570.00,-,-,-
1,2,Other Buildings,122233.47,99000.00,-,-,-
2,3,Other Buildings,171243.00,113950.00,40,-,-
3,4,Other Buildings,74312.98,34419.62,60,-,-
4,5,Other Buildings,205884.00,150000.00,-,2557,1000
...,...,...,...,...,...,...,...
95,96,Hotel and Resort,93314.00,60500.00,-,-,-
96,97,Hotel and Resort,55144.67,25880.00,-,-,-
97,98,Hotel and Resort,53578.62,17373.75,-,-,-
98,99,Hotel and Resort,53499.00,40636.00,-,-,-


In [33]:
building = building.fillna(0)
building = building.replace('-', 0)
building

,building_number,building_type,total_area,cooling_area,solar_power_capacity,ess_capacity,pcs_capacity
0,1,Other Buildings,110634.00,39570.00,0,0,0
1,2,Other Buildings,122233.47,99000.00,0,0,0
2,3,Other Buildings,171243.00,113950.00,40,0,0
3,4,Other Buildings,74312.98,34419.62,60,0,0
4,5,Other Buildings,205884.00,150000.00,0,2557,1000
...,...,...,...,...,...,...,...
95,96,Hotel and Resort,93314.00,60500.00,0,0,0
96,97,Hotel and Resort,55144.67,25880.00,0,0,0
97,98,Hotel and Resort,53578.62,17373.75,0,0,0
98,99,Hotel and Resort,53499.00,40636.00,0,0,0


* pcs 용량 컬럼 삭제
---
pcs 용량은 ess의 속해있는 개념이기 때문에 상관성이 매우 높으므로 제거

In [34]:
building = building.drop('pcs_capacity', axis = 1)
building

,building_number,building_type,total_area,cooling_area,solar_power_capacity,ess_capacity
0,1,Other Buildings,110634.00,39570.00,0,0
1,2,Other Buildings,122233.47,99000.00,0,0
2,3,Other Buildings,171243.00,113950.00,40,0
3,4,Other Buildings,74312.98,34419.62,60,0
4,5,Other Buildings,205884.00,150000.00,0,2557
...,...,...,...,...,...,...
95,96,Hotel and Resort,93314.00,60500.00,0,0
96,97,Hotel and Resort,55144.67,25880.00,0,0
97,98,Hotel and Resort,53578.62,17373.75,0,0
98,99,Hotel and Resort,53499.00,40636.00,0,0


* buildiing_number, building_type category화

In [35]:
building['building_number'] = building['building_number'].astype('category')
building['building_type'] = building['building_type'].astype('category')

## 데이터 병합

In [36]:
train_df = pd.merge(train, building, on = 'building_number', how = 'left' )
test_df = pd.merge(test, building, on = 'building_number', how = 'left')
train_df

,building_number,temperature,windspeed,humidity,power_consumption,day,month,day_of_week,cos_time,sin_time,...,day_hour_mean,hour_mean,hour_std,discomfort,CDH,building_type,total_area,cooling_area,solar_power_capacity,ess_capacity
0,1,18.6,0.9,42.0,1085.28,1,6,2,1.000000,0.000000,...,1371.52,1706.318118,446.882767,63.09388,-7.4,Other Buildings,110634.00,39570.00,0,0
1,1,18.0,1.1,45.0,1047.36,1,6,2,0.965926,0.258819,...,1323.52,1622.620235,439.662704,62.46400,-15.4,Other Buildings,110634.00,39570.00,0,0
2,1,17.7,1.5,45.0,974.88,1,6,2,0.866025,0.500000,...,1236.48,1506.971294,412.071906,62.08735,-23.7,Other Buildings,110634.00,39570.00,0,0
3,1,16.7,1.4,48.0,953.76,1,6,2,0.707107,0.707107,...,1189.92,1437.365647,391.205981,60.89884,-33.0,Other Buildings,110634.00,39570.00,0,0
4,1,18.4,2.8,43.0,986.40,1,6,2,0.500000,0.866025,...,1210.08,1447.321412,381.099697,62.88788,-40.6,Other Buildings,110634.00,39570.00,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203995,100,23.1,0.9,86.0,881.04,24,8,2,0.258819,-0.965926,...,942.96,1010.462118,161.399578,72.38034,-19.6,Hotel and Resort,57497.84,40035.23,0,0
203996,100,22.4,1.3,86.0,798.96,24,8,2,0.500000,-0.866025,...,855.84,928.125176,137.566008,71.21736,-20.2,Hotel and Resort,57497.84,40035.23,0,0
203997,100,21.3,1.0,92.0,825.12,24,8,2,0.707107,-0.707107,...,809.20,830.032941,128.300189,69.79704,-22.3,Hotel and Resort,57497.84,40035.23,0,0
203998,100,21.0,0.3,94.0,640.08,24,8,2,0.866025,-0.500000,...,696.08,723.100235,112.464079,69.41060,-25.1,Hotel and Resort,57497.84,40035.23,0,0


In [37]:
train_df.columns

Index(['building_number', 'temperature', 'windspeed', 'humidity',
       'power_consumption', 'day', 'month', 'day_of_week', 'cos_time',
       'sin_time', 'holiday', 'day_hour_mean', 'hour_mean', 'hour_std',
       'discomfort', 'CDH', 'building_type', 'total_area', 'cooling_area',
       'solar_power_capacity', 'ess_capacity'],
      dtype='object')

In [38]:
train_df.to_csv("pre_train.csv")
test_df.to_csv("pre_test.csv")